<a href="https://colab.research.google.com/github/vitorstaub/corn-classification/blob/main/corn_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificador das principais doenças foliares do milho 🌽
---


*   Cercosporiose  (Cercospora zeae-maydis)
*   Ferrugem Comum
*   Mancha Branca






---
### [Doenças Foliares do Milho - [Embrapa]](https://www.embrapa.br/agencia-de-informacao-tecnologica/cultivos/milho/producao/pragas-e-doencas/doencas/doencas-foliares)


1° Instalar as dependências para a API Gemini que está incluído no pacote google-generativeai

In [1]:
!pip install -q google-generativeai

2° Inicializar o modelo generativo com a API key do Google Ai Studio

In [29]:
from google.colab import userdata
import google.generativeai as genai

GOOGLE_API_KEY=userdata.get('API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

generation_config = {
    'temperature': 0.5,
}

model = genai.GenerativeModel(model_name="gemini-1.0-pro-vision-latest",
                              generation_config=generation_config,)

3° Listar e atribuir o caminho das imagens da pasta de treinamento

In [25]:
import os
import PIL.Image

path = '/training'

def list_images(folder_path):
    images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".png") or f.endswith(".jpg")]
    return sorted(images)

variables = []
images = list_images(path)
for idx, image_path in enumerate(images):
    img_name = f"img{idx}"
    img = PIL.Image.open(image_path)
    print(image_path)
    exec(f"{img_name} = img")
    variables.append(img_name)

print(variables)

/content/img0-cescosporiose.png
/content/img1-mancha-branca.jpg
/content/img2-ferrugem.jpg
/content/img3-cescosporiose.png
/content/img4-ferrugem.png
/content/img5-mancha-branca.png
/content/img6-not.jpg
/content/img7-not.jpg
['img0', 'img1', 'img2', 'img3', 'img4', 'img5', 'img6', 'img7']


# Upload da imagem 🌽🌽🌽
---

4° Fazer  o upload da imagem que se quer identificar (a imagem deve ser da folha do milho, enquanto melhor a imagem mais preciso será)

In [31]:
from google.colab import files

print("Por favor escolha um arquivo para envio:")
uploaded = files.upload()

for fn in uploaded.keys():
  print('Imagem enviada "{name}" com tamanho {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  input = PIL.Image.open(fn)

Por favor escolha um arquivo para envio:


Saving z-cercosporiose-teste.jpg to z-cercosporiose-teste.jpg
Imagem enviada "z-cercosporiose-teste.jpg" com tamanho 49669 bytes


5° Treinamento e resposta do modelo segundo a imagem enviada

In [32]:
prompt_parts = [
  "Você é um modelo de reconhecimento de doenças do milho, com base nos exemplos. o usuário vai mandar uma imagem e você irá retornar o tipo de doença analisando os detalhes da imagem. O output só pode ser de doenças mostradas nos exemplos que são: Cercosporiose do milho, Mancha branca, Ferrugem Comum.",
  "input: Qual a doença dessa folha de milho?",
  img0,
  "output: Cercosporiose  (Cercospora zeae-maydis):\n\nSintomas: \n\nOs sintomas caracterizam-se por manchas de coloração cinza, predominantemente retangulares, com as lesões desenvolvendo-se paralelas às nervuras. Com o desenvolvimento dos sintomas da doença, pode ocorrer necrose de todo o tecido foliar. Em situações de ataques mais severos, as plantas tornam-se mais predispostas às infecções por patógenos no colmo, resultando em maior incidência de acamamento de plantas.\n\nManejo da doença:\n\nRecomenda-se: evitar a permanência de restos da cultura de milho em áreas em que a doença ocorreu com alta severidade para reduzir o inóculo do patógeno na área; realizar a rotação com culturas não hospedeiras como a soja, o sorgo, o girassol, o algodão e outras, uma vez que o milho é o único hospedeiro de C. zeae-maydis; para evitar o aumento do potencial de inóculo de C. zeae-maydis, deve-se evitar o plantio seguido de milho na mesma área; plantar cultivares diferentes em uma mesma área e em cada época de plantio; realizar adubações de acordo com as recomendações técnicas para evitar desequilíbrios nutricionais nas plantas, favoráveis ao desenvolvimento desse patógeno, principalmente a relação nitrogênio/potássio. Para que essas medidas sejam eficientes, recomenda-se a sua aplicação regional (em macrorregiões) para evitar que a doença volte a se manifestar a partir de inóculo trazido pelo vento de lavouras vizinhas infectadas. Em áreas com plantio de cultivares suscetíveis e sob condições ambientais favoráveis para a ocorrência da doença, o controle químico deve ser avaliado como uma opção para o manejo da doença.",
  "input: Qual a doença dessa folha de milho?",
  img1,
  "output: Mancha branca:\n\nAs lesões da mancha branca são, inicialmente, circulares, aquosas e verde claras (anasarcas). Posteriormente, passam a necróticas, de cor palha, circulares a elípticas, com diâmetro variando de 0,3 cm a 1 cm. Geralmente, são encontradas dispersas no limbo foliar, mas iniciam-se na ponta da folha progredindo para a base, podendo coalescer. Em geral, os sintomas aparecem inicialmente nas folhas inferiores, progredindo rapidamente para as superiores, sendo mais severos após o pendoamento. Sob condições de ataque severo, os sintomas da doença podem ser observados também na palha da espiga. Em condições de campo, os sintomas não ocorrem, normalmente, em plântulas de milho.",
  "input: Qual a doença dessa folha de milho?",
  img2,
  "output: Ferrugem Comum:\n\nA ferrugem comum caracteriza-se pela formação de pústulas em toda a parte aérea da planta, mas com maior abundância nas folhas. As pústulas ocorrem em ambas as superfícies da folha, sendo esta uma das características que a diferencia da ferrugem polissora, cujas pústulas predominam na superfície superior da folha. As pústulas da ferrugem comum apresentam formato circular a alongado e coloração castanho clara a escuro, que se acentua à medida em que as pústulas amadurecem e se rompem, liberando os uredósporos, que são os esporos típicos do patógeno. Sob condições ambientais favoráveis, as pústulas podem coalescer, formando grandes áreas necróticas nas folhas.",
  "input: Qual a doença dessa folha de milho?",
  img3,
  "output: Cercosporiose  (Cercospora zeae-maydis):\n\nSintomas: \n\nOs sintomas caracterizam-se por manchas de coloração cinza, predominantemente retangulares, com as lesões desenvolvendo-se paralelas às nervuras. Com o desenvolvimento dos sintomas da doença, pode ocorrer necrose de todo o tecido foliar. Em situações de ataques mais severos, as plantas tornam-se mais predispostas às infecções por patógenos no colmo, resultando em maior incidência de acamamento de plantas.\n\nManejo da doença:\n\nRecomenda-se: evitar a permanência de restos da cultura de milho em áreas em que a doença ocorreu com alta severidade para reduzir o inóculo do patógeno na área; realizar a rotação com culturas não hospedeiras como a soja, o sorgo, o girassol, o algodão e outras, uma vez que o milho é o único hospedeiro de C. zeae-maydis; para evitar o aumento do potencial de inóculo de C. zeae-maydis, deve-se evitar o plantio seguido de milho na mesma área; plantar cultivares diferentes em uma mesma área e em cada época de plantio; realizar adubações de acordo com as recomendações técnicas para evitar desequilíbrios nutricionais nas plantas, favoráveis ao desenvolvimento desse patógeno, principalmente a relação nitrogênio/potássio. Para que essas medidas sejam eficientes, recomenda-se a sua aplicação regional (em macrorregiões) para evitar que a doença volte a se manifestar a partir de inóculo trazido pelo vento de lavouras vizinhas infectadas. Em áreas com plantio de cultivares suscetíveis e sob condições ambientais favoráveis para a ocorrência da doença, o controle químico deve ser avaliado como uma opção para o manejo da doença.",
  "input: Qual a doença dessa folha de milho?",
  img4,
  "output: Ferrugem Comum:\n\nA ferrugem comum caracteriza-se pela formação de pústulas em toda a parte aérea da planta, mas com maior abundância nas folhas. As pústulas ocorrem em ambas as superfícies da folha, sendo esta uma das características que a diferencia da ferrugem polissora, cujas pústulas predominam na superfície superior da folha. As pústulas da ferrugem comum apresentam formato circular a alongado e coloração castanho clara a escuro, que se acentua à medida em que as pústulas amadurecem e se rompem, liberando os uredósporos, que são os esporos típicos do patógeno. Sob condições ambientais favoráveis, as pústulas podem coalescer, formando grandes áreas necróticas nas folhas.",
  "input: Qual a doença dessa folha de milho?",
  img5,
  "output: Mancha branca:\n\nAs lesões da mancha branca são, inicialmente, circulares, aquosas e verde claras (anasarcas). Posteriormente, passam a necróticas, de cor palha, circulares a elípticas, com diâmetro variando de 0,3 cm a 1 cm. Geralmente, são encontradas dispersas no limbo foliar, mas iniciam-se na ponta da folha progredindo para a base, podendo coalescer. Em geral, os sintomas aparecem inicialmente nas folhas inferiores, progredindo rapidamente para as superiores, sendo mais severos após o pendoamento. Sob condições de ataque severo, os sintomas da doença podem ser observados também na palha da espiga. Em condições de campo, os sintomas não ocorrem, normalmente, em plântulas de milho.",
  "input: Qual a doença dessa folha de milho?",
  img6,
  "output: Folha de milho saudavel",
  "input: Qual a doença dessa folha de milho?",
  img7,
  "output: Folha de milho saudavel",
  "input: Qual a doença dessa folha de milho?",
  input,
  "output: ",
]

response = model.generate_content(prompt_parts)
print(response.text)

 Cercosporiose  (Cercospora zeae-maydis):

Sintomas: 

Os sintomas caracterizam-se por manchas de coloração cinza, predominantemente retangulares, com as lesões desenvolvendo-se paralelas às nervuras. Com o desenvolvimento dos sintomas da doença, pode ocorrer necrose de todo o tecido foliar. Em situações de ataques mais severos, as plantas tornam-se mais predispostas às infecções por patógenos no colmo, resultando em maior incidência de acamamento de plantas.

Manejo da doença:

Recomenda-se: evitar a permanência de restos da cultura de milho em áreas em que a doença ocorreu com alta severidade para reduzir o inóculo do patógeno na área; realizar a rotação com culturas não hospedeiras como a soja, o sorgo, o girassol, o algodão e outras, uma vez que o milho é o único hospedeiro de C. zeae-maydis; para evitar o aumento do potencial de inóculo de C. zeae-maydis, deve-se evitar o plantio seguido de milho na mesma área; plantar cultivares diferentes em uma mesma área e em cada época de pla

# Deixando o texto mais bonito
---

In [33]:
from IPython.display import display
from IPython.display import Markdown
import textwrap

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

to_markdown(response.text)

>  Cercosporiose  (Cercospora zeae-maydis):
> 
> Sintomas: 
> 
> Os sintomas caracterizam-se por manchas de coloração cinza, predominantemente retangulares, com as lesões desenvolvendo-se paralelas às nervuras. Com o desenvolvimento dos sintomas da doença, pode ocorrer necrose de todo o tecido foliar. Em situações de ataques mais severos, as plantas tornam-se mais predispostas às infecções por patógenos no colmo, resultando em maior incidência de acamamento de plantas.
> 
> Manejo da doença:
> 
> Recomenda-se: evitar a permanência de restos da cultura de milho em áreas em que a doença ocorreu com alta severidade para reduzir o inóculo do patógeno na área; realizar a rotação com culturas não hospedeiras como a soja, o sorgo, o girassol, o algodão e outras, uma vez que o milho é o único hospedeiro de C. zeae-maydis; para evitar o aumento do potencial de inóculo de C. zeae-maydis, deve-se evitar o plantio seguido de milho na mesma área; plantar cultivares diferentes em uma mesma área e em cada época de plantio; realizar adubações de acordo com as recomendações técnicas para evitar desequilíbrios nutricionais nas plantas, favoráveis ao desenvolvimento desse patógeno, principalmente a relação nitrogênio/potássio. Para que essas medidas sejam eficientes, recomenda-se a sua aplicação regional (em macrorregiões) para evitar que a doença volte a se manifestar a partir de inóculo trazido pelo vento de lavouras vizinhas infectadas. Em áreas com plantio de cultivares suscetíveis e sob condições ambientais favoráveis para a ocorrência da doença, o controle químico deve ser avaliado como uma opção para o manejo da doença.